# Cycling data 3. Maps - plot .fit file on map using folium

### Contents
0. Install fitparse and python libraries
1. Parse the fit file and read into Pandas dataframe
2. Data cleaning
3. Show data on a map using folium
4. Show a map using GeoPandas


#### To do:
- [ ] issue folium
- [ ] convert points to polyline (https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/)
- [ ] polyline/points different colors based on value (https://stackoverflow.com/questions/42756934/how-to-plot-lat-and-long-from-pandas-dataframe-on-folium-map-group-by-some-label)
- [ ] also to find start/endpoint based on given point
- [ ] show segment in table + on map


## 0. Install fitparse and import the python libraries

We use the fitparse library to parse the .fit file.

Please find the github of fitparse project here: http://dtcooper.github.io/python-fitparse/

In [1]:
!pip install fitparse 

In [2]:
#import the modules
import numpy as np
import pandas as pd
import fitparse

## 1. import the .fit file and read it into pandas dataframe


In [1]:
# parse the fit file and read it into a list called workout
#source: http://johannesjacob.com/
from fitparse import FitFile
import pandas as pd
import matplotlib.pyplot as plt

fitfile = FitFile('3919172E.fit') ### enter the name of the file here!

while True:
    try:
        fitfile.messages
        break
    except KeyError:
        continue
workout = []
for record in fitfile.get_messages('record'):
    r = {}
    for record_data in record:
        r[record_data.name] = record_data.value
    workout.append(r)

In [2]:
#read this list into a pandas dataframe
fietsdata = pd.DataFrame(workout)
fietsdata.head()

,accumulated_power,altitude,cadence,calories,distance,enhanced_altitude,enhanced_speed,grade,heart_rate,left_pedal_smoothness,left_right_balance,left_torque_effectiveness,position_lat,position_long,power,right_pedal_smoothness,right_torque_effectiveness,speed,temperature,timestamp
0,9241.0,108.0,64.0,0,0.0,108.0,5.926,None,115,None,None,None,624663697.0,57981701.0,111.0,None,None,5.926,23,2020-05-09 08:00:47
1,9370.0,108.0,65.0,0,12.0,108.0,6.471,None,117,None,None,None,624663458.0,57980727.0,129.0,None,None,6.471,23,2020-05-09 08:00:48
2,9370.0,107.0,65.0,0,12.0,107.0,6.327,None,119,None,None,None,624663200.0,57979693.0,112.0,None,None,6.327,23,2020-05-09 08:00:49
3,9482.0,107.0,65.0,0,24.0,107.0,6.214,None,120,None,None,None,624662921.0,57978719.0,132.0,None,None,6.214,23,2020-05-09 08:00:50
4,9674.0,106.0,66.0,1,24.0,106.0,6.049,None,121,None,None,None,624662723.0,57977665.0,60.0,None,None,6.049,23,2020-05-09 08:00:51


In [3]:
#check data
fietsdata.dtypes

accumulated_power                    float64
altitude                             float64
cadence                              float64
calories                               int64
distance                             float64
enhanced_altitude                    float64
enhanced_speed                       float64
grade                                 object
heart_rate                             int64
left_pedal_smoothness                 object
left_right_balance                    object
left_torque_effectiveness             object
position_lat                         float64
position_long                        float64
power                                float64
right_pedal_smoothness                object
right_torque_effectiveness            object
speed                                float64
temperature                            int64
timestamp                     datetime64[ns]
dtype: object

## 2. Data cleaning: convert semicircles to degrees

Bike computers store geodata commonly as semicircles as this improve storage capacity. 
To use this data, we need to convert it from semicircles to degrees.

We can use the following formula: degrees = semicircles * (180 / 2 ** 31)

In [4]:
#Convert position_long and position_lat from semicircles to degrees
fietsdata['pos_lat'] = fietsdata['position_lat']*  (180 / 2**31)
fietsdata['pos_long'] = fietsdata['position_long']*  (180 / 2**31)
fietsdata.head()

,accumulated_power,altitude,cadence,calories,distance,enhanced_altitude,enhanced_speed,grade,heart_rate,left_pedal_smoothness,...,position_lat,position_long,power,right_pedal_smoothness,right_torque_effectiveness,speed,temperature,timestamp,pos_lat,pos_long
0,9241.0,108.0,64.0,0,0.0,108.0,5.926,None,115,None,...,624663697.0,57981701.0,111.0,None,None,5.926,23,2020-05-09 08:00:47,52.358706,4.859970
1,9370.0,108.0,65.0,0,12.0,108.0,6.471,None,117,None,...,624663458.0,57980727.0,129.0,None,None,6.471,23,2020-05-09 08:00:48,52.358686,4.859888
2,9370.0,107.0,65.0,0,12.0,107.0,6.327,None,119,None,...,624663200.0,57979693.0,112.0,None,None,6.327,23,2020-05-09 08:00:49,52.358665,4.859802
3,9482.0,107.0,65.0,0,24.0,107.0,6.214,None,120,None,...,624662921.0,57978719.0,132.0,None,None,6.214,23,2020-05-09 08:00:50,52.358641,4.859720
4,9674.0,106.0,66.0,1,24.0,106.0,6.049,None,121,None,...,624662723.0,57977665.0,60.0,None,None,6.049,23,2020-05-09 08:00:51,52.358625,4.859632


## 3. Show data on map using Folium

### 3a. Use folium to create a base map

In [6]:
# uncomment when you've never used folium before
%pip install folium   

Note: you may need to restart the kernel to use updated packages.


In [7]:
import folium

cycle_map = folium.Map(location=[52.373,4.890],  #amsterdammers believe their city is the center of the world but feel free to change the coordinates to your liking ;-)
                  tiles='Stamen Terrain', # choose Stamen Terrain as base map or use another
                  zoom_start=10, # zoom level. Lower = zoom out; higher = zoom in
                  detect_retina=True) # optional, used to detect high quality screes

cycle_map

### 3b. Remove all unnecessary data; just keep the lat, long.

In [8]:
# print the columns
fietsdata.columns

Index(['accumulated_power', 'altitude', 'cadence', 'calories', 'distance',
       'enhanced_altitude', 'enhanced_speed', 'grade', 'heart_rate',
       'left_pedal_smoothness', 'left_right_balance',
       'left_torque_effectiveness', 'position_lat', 'position_long', 'power',
       'right_pedal_smoothness', 'right_torque_effectiveness', 'speed',
       'temperature', 'timestamp', 'pos_lat', 'pos_long'],
      dtype='object')

In [11]:
# remove all columns except 'pos_lat' and 'pos_long'
coor = fietsdata.drop(['accumulated_power', 'altitude', 'cadence', 'calories', 'distance',
       'enhanced_altitude', 'enhanced_speed', 'grade', 'heart_rate',
       'left_pedal_smoothness', 'left_right_balance',
       'left_torque_effectiveness', 'position_lat', 'position_long', 'power',
       'right_pedal_smoothness', 'right_torque_effectiveness', 'speed',
       'temperature', 'timestamp'], axis=1) # drop NaN
coor_valid = coor[coor.pos_lat != 'None'] # drop None
coor_valid

,pos_lat,pos_long
0,52.358706,4.859970
1,52.358686,4.859888
2,52.358665,4.859802
3,52.358641,4.859720
4,52.358625,4.859632
...,...,...
14906,52.359196,4.861780
14907,52.359190,4.861783
14908,52.359188,4.861792
14909,52.359188,4.861797


### 3c. Plot the coordinates on the map

In [12]:
import folium

coor_valid.apply(lambda row:folium.Circle(radius=10, location=[row["pos_lat"], 
                                                  row["pos_long"]]).add_to(cycle_map),
                axis=1)
cycle_map.add_child(folium.LatLngPopup())
cycle_map

ValueError: Location values cannot contain NaNs.

In [13]:
cycle_map.save('cycle_map.html') # save the map as an html file

## 4. Segments  - WORK IN PROGRESS

Use the map above to select a start point and endpoint close to your route.

Code: 
Input:
-  enter start point Latitude: 52.3559 Longitude: 4.9681
-  enter last point Latitude: 52.3570 Longitude: 4.9706

Logic:
- find nearest points on route based on point class example for start/end points

Return
- Return data for this segment:
    - time, speed, heart_rate, power, cadence
- show in graph


In [ ]:
# Input: ask the user to specify the segment.

segment_name = input("Enter name of segment")

startpoint_lat = float(input("Enter lat of startpoint"))
startpoint_long = float(input("Enter long of startpoint"))

endpoint_lat = float(input("Enter lat of startpoint"))
endpoint_long = float(input("Enter long of end point"))

print(segment_name)
print("startpoint="+ str(startpoint_lat) +str(startpoint_long))
print("endpoint=" + str(endpoint_lat) + str(endpoint_long))


Enter name of segmentmihiel
